In [1]:
from dask.distributed import Client, progress

client = Client("tcp://192.168.254.34:8786")
# client = Client(processes=False, threads_per_worker=3, n_workers=4, memory_limit="10GB")
# client = Client(n_workers=4)

client

Client Scheduler: tcp://192.168.254.34:8786 Dashboard: http://192.168.254.34:8787/status,Cluster Workers: 2 Cores: 24 Memory: 53.34 GB


In [2]:
from dask_cassandra_loader import Loader

dask_cassandra_loader = Loader()


###  cassandra ornegi çalıtırma

wget https://dask-cassandra-loader.readthedocs.io/en/latest/_downloads/5f6a89aeb41f1a7e7bdcb0b51bd37a76/tutorial.cql

docker run --name cassandra-server -p 9042:9042 -d -v bitcassvol:/bitnami bitnami/cassandra:latest

docker run --name castest -it --rm bitnami/cassandra:latest bash

#### hostta
docker cp tutorial.cql castest:/

#### castest container'ında
HOST_IP=10.151.255.101
cqlsh -u cassandra -p cassandra $HOST_IP -e "create keyspace tutorial with replication = {'class': 'SimpleStrategy', 'replication_factor': 1};"

cqlsh -u cassandra -p cassandra $HOST_IP --keyspace=tutorial -f tutorial.cql


In [3]:
HOST_IP="10.151.255.101"
HOST_IP="10.151.255.101"
keyspace = 'tutorial'
cluster = [HOST_IP]

In [4]:
dask_cassandra_loader.connect_to_cassandra(cluster,
                                           keyspace,
                                           username='cassandra',
                                           password='cassandra')

dask_cassandra_loader.connect_to_local_dask()

In [3]:
cluster = "192.168.254.34:8786"
dask_cassandra_loader.connect_to_dask(cluster)

In [ ]:
table = dask_cassandra_loader.load_cassandra_table('tab1',
                                         ['id', 'year', 'month', 'day'],
                                         [('day', 'equal', [8])],
                                         [('id', [18]), ('year', [2018]),
                                          ('month', [11])],
                                         force=False)

In [ ]:
if table is None:
    raise AssertionError("Table is not supposed to be None!!!")

if table.data is None:
    raise AssertionError("Table.data is not supposed to be None!!!")

In [ ]:
local_table = table.data.compute()

In [ ]:
print(local_table.head())

In [ ]:
table2 = dask_cassandra_loader.load_cassandra_table('tab1',
                                         ['id', 'year', 'month', 'day'],[],[],
                                         force=True)

In [ ]:
local_table2 = table2.data.compute()

In [ ]:
local_table2.head()

table3 = dask_cassandra_loader.load_cassandra_table('cylinder_target_2',
                                         ['personid', 'status', 'finger_index'],[],[],
                                         force=True)

In [8]:
table3 = dask_cassandra_loader.load_cassandra_table('dnm',
                                         ['personid', 'status', 'finger_index', 'cylindersbinary'],[],[],
                                         force=True)

In [9]:
local_table3 = table3.data.compute()

In [14]:
local_table3.tail()

,personid,status,finger_index,cylindersbinary
8,413208,1,19,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
9,413208,1,20,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
0,186584,1,11,b'\x00\xff\xff\x00\xf0\xff\xff\x00\xff\xff\xff...
1,186584,1,12,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xf0\xff\xff...
2,186584,1,13,b'\x00\x00\xf0\x00\x00\x00\xf0\x0f\x00\x00\xff...


del local_table3

In [12]:
type(local_table3)

pandas.core.frame.DataFrame

In [13]:
type(table3)

dask_cassandra_loader.loader.Table

In [16]:
import pandas as pd

pdf = pd.DataFrame(data={'col1': [1, 2], 'col2': [3, 4]})

pip install pyarrow#  veya fastparquet

In [24]:
local_table3.to_parquet('local_table3.parquet.gzip',
              compression='gzip')

In [26]:
pd.read_parquet('local_table3.parquet.gzip')

,personid,status,finger_index,cylindersbinary
0,365538,1,11,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
1,365538,1,12,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
2,365538,1,13,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
3,365538,1,14,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
4,365538,1,15,b'\x00\x00\xff\x00\x00\x00\xff\x0f\x00\xf0\xff...
...,...,...,...,...
8,413208,1,19,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
9,413208,1,20,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
0,186584,1,11,b'\x00\xff\xff\x00\xf0\xff\xff\x00\xff\xff\xff...
1,186584,1,12,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xf0\xff\xff...
